In [1]:
#Libraries needed
import re
import time
import csv
import yake
import nltk  
import neointerface
from neo4j import GraphDatabase
from getpass import getpass
from time import sleep
from py2neo import Graph,Node,Relationship
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge, EdgeOptions 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer  
from nltk.corpus import stopwords  
from nltk import PorterStemmer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
#Clean text into tokens
def clean_text(text):
    text=text.lower()
    tokens=tokenizer.tokenize(str(text))
    new_tokens=[token for token in tokens if token not in en_stopwords]
    stemmed_tokens=[ps.stem(tokens) for tokens in new_tokens]
    clean_text=" ".join(stemmed_tokens)
    return clean_text

In [3]:
#Extract top 20 key phrases from a given text block
def key_phrase(text):
    kw_extractor = yake.KeywordExtractor(top=20, stopwords=None)
    keywords = kw_extractor.extract_keywords(text)
    phrases=[]
    for kw in range(len(keywords)):
        phrases.append(keywords[kw][0])
    return phrases

In [4]:
#Perform Sentiment Analysis on a given text block and assign a label
def sen_analy(sentence):
    score = analyser.polarity_scores(sentence)
    label='Neutral'
    if(score['neg']>0 and score['neg']>score['pos'] and score['compound']<0):
        label='Complaint'
        return label
    elif(score['pos']>0 and score['neg']<score['pos'] and score['compound']>0):
        label='Appreciation'
        return label
    else:
        return label

In [5]:
#Extract values from a tweet card
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element('xpath','.//span').text
    try:
        handle = card.find_element('xpath','.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element('xpath','.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    text = str(card.find_element('xpath','.//div[2]/div[2]/div[2]').text)
    reply_cnt = card.find_element('xpath','.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element('xpath','.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element('xpath','.//div[@data-testid="like"]').text

    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet

In [27]:
# create instance of web driver
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options,executable_path='C:/Users/Aibhinav/Edgedriver/msedgedriver.exe')
search_term=input('Enter the products to scrape data on :')

C:\Users\Aibhinav\AppData\Local\Temp\ipykernel_22468\603581446.py:4: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(options=options,executable_path='C:/Users/Aibhinav/Edgedriver/msedgedriver.exe')


Enter the products to scrape data on :Hp printers


In [29]:
# Sign into twitter using Personal Credentials.
# Automation using Selenium has been blocked by twitter
driver.get('https://twitter.com/login')

In [30]:
# navigate to login screen
driver.get('https://twitter.com/search')
driver.maximize_window()
sleep(5)

# find search input and search for term
search_input = driver.find_element('xpath','//input[@aria-label="Search query"]')
search_input.send_keys(search_term)
search_input.send_keys(Keys.RETURN)
sleep(1)

# navigate to historical 'latest' tab
driver.find_element('link text','Latest').click()

In [31]:
#Declare variable to store tweet data
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements('xpath','//article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break
#Scroll up to stop scraping and ignore the below error block

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: MicrosoftEdge=113.0.1774.42)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00007FF6A8FC3472+67490]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF6A8F59402+786594]
	(No symbol) [0x00007FF6A8CFC646]
	(No symbol) [0x00007FF6A8D016B2]
	(No symbol) [0x00007FF6A8D03364]
	(No symbol) [0x00007FF6A8D03450]
	(No symbol) [0x00007FF6A8D3F4F6]
	(No symbol) [0x00007FF6A8D3FB8A]
	(No symbol) [0x00007FF6A8D337CC]
	(No symbol) [0x00007FF6A8D5DF8F]
	(No symbol) [0x00007FF6A8D33751]
	(No symbol) [0x00007FF6A8D33621]
	(No symbol) [0x00007FF6A8D5E280]
	(No symbol) [0x00007FF6A8D33751]
	(No symbol) [0x00007FF6A8D76BC5]
	(No symbol) [0x00007FF6A8D5DD23]
	(No symbol) [0x00007FF6A8D32794]
	(No symbol) [0x00007FF6A8D319B0]
	(No symbol) [0x00007FF6A8D32F04]
	Microsoft::Applications::Events::ILogManager::DispatchEventBroadcast [0x00007FF6A918EC83+1309699]
	(No symbol) [0x00007FF6A8

In [32]:
#Run this code block to end the scraping of tweets and close the driver
driver.close()

In [8]:
#Show the amount of tweets scraped
len(data)

NameError: name 'data' is not defined

In [34]:
#Run this code block to save the tweets scraped
with open('Hpprinter_tweets.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Comments', 'Likes', 'Retweets']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)

In [6]:
#Read the file and store in a file
uncleaned_tweets=pd.read_csv('Hpprinter_tweets.csv')

In [7]:
uncleaned_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   UserName   280 non-null    object 
 1   Handle     282 non-null    object 
 2   Timestamp  282 non-null    object 
 3   Text       282 non-null    object 
 4   Comments   51 non-null     float64
 5   Likes      27 non-null     float64
 6   Retweets   89 non-null     float64
dtypes: float64(3), object(4)
memory usage: 15.5+ KB


In [8]:
#Fill NA values with 0 for Likes,Tweets and Retweet counts
uncleaned_tweets["Comments"].fillna(0, inplace = True)
uncleaned_tweets["Likes"].fillna(0, inplace = True)
uncleaned_tweets["Retweets"].fillna(0, inplace = True)

In [9]:
uncleaned_tweets

,UserName,Handle,Timestamp,Text,Comments,Likes,Retweets
0,benwokes014,@benwokes014,2023-05-17T09:06:23.000Z,"https://youtu.be/tmSasSVCVdI How To Print, Sca...",0.0,0.0,1.0
1,benwokes014,@benwokes014,2023-05-17T09:04:43.000Z,https://youtube.com/shorts/cxSxpVSXLaI… HP Pri...,0.0,0.0,1.0
2,benwokes014,@benwokes014,2023-05-17T09:03:11.000Z,"https://youtu.be/PMvIgso2c6I HP DeskJet 4100, ...",0.0,0.0,1.0
3,László Babus,@MatrixfanLive,2023-05-17T08:59:27.000Z,HP disables printers if you use third-party in...,0.0,0.0,0.0
4,Nadia.HilllEiv,@hillleiv,2023-05-17T08:39:37.000Z,Icehtank Printer Ink Cartridge 364XL 364 XL Re...,0.0,0.0,0.0
...,...,...,...,...,...,...,...
277,Geeky Bargain,@GeekyBargain,2023-05-14T17:11:55.000Z,HP Wireless Color All-in-One Printer with 6 Mo...,0.0,0.0,0.0
278,Mike Marshall,@bearpoop,2023-05-14T16:39:37.000Z,@HP\n They said that only some of its printe...,1.0,0.0,0.0
279,Mike Marshall,@bearpoop,2023-05-14T16:39:09.000Z,They said that only some of its printers are e...,0.0,0.0,0.0
280,Burt May,@burtmay,2023-05-14T16:29:59.000Z,#brotherprinters. I finally had enough of this...,0.0,0.0,0.0


In [10]:
#Declare tokenizer and stopwords for cleaning
tokenizer=RegexpTokenizer(r'\w+')  
en_stopwords=set(stopwords.words('english'))  
ps=PorterStemmer() 

In [11]:
#Convert the Text datatype in Dataframe to string for cleaning
uncleaned_tweets['Text']=uncleaned_tweets['Text'].astype('string')
print(uncleaned_tweets.dtypes)

UserName      object
Handle        object
Timestamp     object
Text          string
Comments     float64
Likes        float64
Retweets     float64
dtype: object


In [12]:
#Cleaning the text of tweets
for i in range(len(uncleaned_tweets)):
    uncleaned_tweets.loc[i,'clean_text']=clean_text(str(uncleaned_tweets.loc[i,'Text']))

In [13]:
cleaned_tweets=uncleaned_tweets

In [14]:
#Declaring the Sentiment Analyzer
analyser = SentimentIntensityAnalyzer()

In [15]:
#Categorizing the data based on Sentimenets
for i in range(len(cleaned_tweets)):
    cleaned_tweets.loc[i,'category']=sen_analy(cleaned_tweets.loc[i,'Text'])

In [16]:
#Extracting Key phrases for every tweet
prod_phrases=[]
for i in range(len(cleaned_tweets)):
    phrase=key_phrase(cleaned_tweets.loc[i,'Text'])
    prod_phrases.append(phrase)
print(prod_phrases)

[['Smart App', 'Scan', 'Print', 'Copy', 'Deskjet', 'App', 'Printer', 'Smart', 'Computer', 'Drivers', 'Software'], ['Connect to Wi-Fi', 'Printer Connect', 'Install', 'Computer', 'Drivers', 'Software', 'Deskjet', 'Printer', 'Connect', 'Wi-Fi'], ['USB WiFi SetUp', 'WiFi SetUp Windows', 'USB WiFi', 'SetUp Windows', 'USB', 'Windows', 'Printer Setup', 'WiFi SetUp', 'Printer', 'Computer', 'Drivers', 'Software', 'DeskJet', 'SetUp', 'WiFi'], ['third-party ink cartridges', 'ink cartridges', 'disables printers', 'third-party ink', 'cartridges', 'disables', 'printers', 'third-party', 'ink', 'Big', 'Color runs dry', 'cks', 'Color runs', 'quickly despite printing', 'n’t buy', 'Color', 'printing', 'footer', 'small company logos', 'runs dry'], ['Printer Ink Cartridge', 'Icehtank Printer Ink', 'Ink Cartridge', 'Printer Ink', 'Icehtank Printer', 'Deskjet', 'Cartridge', 'Photosmart', 'Printer', 'Ink', 'Replace', 'Icehtank'], ['third-party ink cartridges', 'ink cartridges', 'disables printers', 'third-par

In [17]:
#Storing the Keyphrases of tweets into the dataframe
cleaned_tweets['Key_phrases'] = [[] for r in range(len(cleaned_tweets))]
for i in range(len(cleaned_tweets)):
    cleaned_tweets.loc[i, 'Key_phrases'][0:0]=prod_phrases[i]

In [18]:
print(cleaned_tweets)

             UserName          Handle                 Timestamp  \
0         benwokes014    @benwokes014  2023-05-17T09:06:23.000Z   
1         benwokes014    @benwokes014  2023-05-17T09:04:43.000Z   
2         benwokes014    @benwokes014  2023-05-17T09:03:11.000Z   
3        László Babus  @MatrixfanLive  2023-05-17T08:59:27.000Z   
4      Nadia.HilllEiv       @hillleiv  2023-05-17T08:39:37.000Z   
..                ...             ...                       ...   
277     Geeky Bargain   @GeekyBargain  2023-05-14T17:11:55.000Z   
278     Mike Marshall       @bearpoop  2023-05-14T16:39:37.000Z   
279     Mike Marshall       @bearpoop  2023-05-14T16:39:09.000Z   
280          Burt May        @burtmay  2023-05-14T16:29:59.000Z   
281  Cloud DevOps Man   @Cloud_Devops  2023-05-14T16:10:00.000Z   

                                                  Text  Comments  Likes  \
0    https://youtu.be/tmSasSVCVdI How To Print, Sca...       0.0    0.0   
1    https://youtube.com/shorts/cxSxpVSXLaI… 

In [20]:
labelled_tweets=cleaned_tweets
labelled_tweets.to_csv('labelled_data.csv')

In [21]:
#Assigning an ID to every tweet
labelled_tweets['ID'] = labelled_tweets.reset_index().index
print(labelled_tweets)

             UserName          Handle                 Timestamp  \
0         benwokes014    @benwokes014  2023-05-17T09:06:23.000Z   
1         benwokes014    @benwokes014  2023-05-17T09:04:43.000Z   
2         benwokes014    @benwokes014  2023-05-17T09:03:11.000Z   
3        László Babus  @MatrixfanLive  2023-05-17T08:59:27.000Z   
4      Nadia.HilllEiv       @hillleiv  2023-05-17T08:39:37.000Z   
..                ...             ...                       ...   
277     Geeky Bargain   @GeekyBargain  2023-05-14T17:11:55.000Z   
278     Mike Marshall       @bearpoop  2023-05-14T16:39:37.000Z   
279     Mike Marshall       @bearpoop  2023-05-14T16:39:09.000Z   
280          Burt May        @burtmay  2023-05-14T16:29:59.000Z   
281  Cloud DevOps Man   @Cloud_Devops  2023-05-14T16:10:00.000Z   

                                                  Text  Comments  Likes  \
0    https://youtu.be/tmSasSVCVdI How To Print, Sca...       0.0    0.0   
1    https://youtube.com/shorts/cxSxpVSXLaI… 

In [22]:
#Dropping clean_text column as it is not needed for knowledge graph nodes 
labelled_tweets=labelled_tweets.drop(['clean_text'],axis=1)
print(labelled_tweets)

             UserName          Handle                 Timestamp  \
0         benwokes014    @benwokes014  2023-05-17T09:06:23.000Z   
1         benwokes014    @benwokes014  2023-05-17T09:04:43.000Z   
2         benwokes014    @benwokes014  2023-05-17T09:03:11.000Z   
3        László Babus  @MatrixfanLive  2023-05-17T08:59:27.000Z   
4      Nadia.HilllEiv       @hillleiv  2023-05-17T08:39:37.000Z   
..                ...             ...                       ...   
277     Geeky Bargain   @GeekyBargain  2023-05-14T17:11:55.000Z   
278     Mike Marshall       @bearpoop  2023-05-14T16:39:37.000Z   
279     Mike Marshall       @bearpoop  2023-05-14T16:39:09.000Z   
280          Burt May        @burtmay  2023-05-14T16:29:59.000Z   
281  Cloud DevOps Man   @Cloud_Devops  2023-05-14T16:10:00.000Z   

                                                  Text  Comments  Likes  \
0    https://youtu.be/tmSasSVCVdI How To Print, Sca...       0.0    0.0   
1    https://youtube.com/shorts/cxSxpVSXLaI… 

In [25]:
#Converting ID and Key phrases to a separate CSV file for evalutation
phr={}
for i in range(len(labelled_tweets)):
    for j in labelled_tweets.loc[i, 'Key_phrases']:
        if j in phr:
            phr[j].append(int(labelled_tweets.loc[i,'ID']))
        else:
            phr[j]=[int(labelled_tweets.loc[i,'ID'])]

In [26]:
#Storing in dataframe and then to CSV file
df = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in phr.items()])).T
df.index.name = 'Phrase'
df.reset_index(inplace=True)
df.replace(np.nan, -1, inplace=True)
df.to_csv('phrases.csv')

In [134]:
db = neointerface.NeoInterface(host="neo4j://localhost:7687" , credentials=("YOUR USERNAME", "YOUR PASSWORD"))

---------------- Initializing NeoInterface -------------------
Connection to neo4j://localhost:7687 established


In [80]:
#Loading the dataframe into the database as nodes
db.load_df(label,"labelled_tweets")

[281,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,


In [86]:
#Cypher queries to build more nodes and relationships
with driver.session(database='neo4j') as session:
    graph=session.run('MATCH (n) RETURN n').graph()

In [95]:
#Assigning nodes for various sentiments
with driver.session(database='neo4j') as session:
    graph=session.run("CREATE (n:Sentiment {sentiment: 'Appreciation'})").graph()

In [96]:
with driver.session(database='neo4j') as session:
    graph=session.run("CREATE (n:Sentiment {sentiment: 'Neutral'})").graph()

In [97]:
with driver.session(database='neo4j') as session:
    graph=session.run("CREATE (n:Sentiment {sentiment: 'Complaint'})").graph()

In [98]:
#Assigning relationship between tweet and sentiment nodes
with driver.session(database='neo4j') as session:
    graph=session.run("MATCH (a:labelled_tweets),(b:Sentiment) WHERE a.category = b.sentiment CREATE (a)-[r:OF]->(b)").graph()

In [104]:
#Building nodes for Replies and Original tweets
with driver.session(database='neo4j') as session:
    graph=session.run("CREATE (n:Type {category: 'Replies'})").graph()

In [105]:
with driver.session(database='neo4j') as session:
    graph=session.run("CREATE (n:Type {category: 'Original'})").graph()

In [107]:
#Assigning relationship between Type of tweet and tweets
with driver.session(database='neo4j') as session:
    graph=session.run("MATCH (a:labelled_tweets WHERE a.Text STARTS WITH 'Replying to'),(b:Type WHERE b.category = 'Replies') MERGE (a)-[r:type]->(b)").graph()
    

In [108]:
with driver.session(database='neo4j') as session:
    graph=session.run("MATCH (a:labelled_tweets WHERE NOT a.Text STARTS WITH 'Replying to'),(b:Type WHERE b.category = 'Original') MERGE (a)-[r:type]->(b)").graph()
    